In [51]:
!pip install Flask-RESTful

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 52.8/52.8 kB 390.0 kB/s eta 0:00:00


In [50]:
!pip install Flask

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import torch
import clip
import cv2
import numpy as np
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [35]:
THRESHOLD = 0.27
NUM_FRAMES = 5

In [19]:
def check(score):
    curr_score = 0
    i = 0
    cnt = 0
    while i < NUM_FRAMES and i < len(score):
        curr_score += score[i]
        i += 1
    if(curr_score/NUM_FRAMES >= THRESHOLD):
        cnt+=1
    while i < len(score):
        curr_score += score[i];
        curr_score -= score[i- NUM_FRAMES]
        if(curr_score/NUM_FRAMES >= THRESHOLD):
            cnt+=1
        i+=1
    return cnt

In [23]:
def process_videos(video_paths):
    res = []
    for video_path in video_paths:
        frames = extract_frames(video_path)
        curr_vid = []
        for img in frames:
            image = preprocess(img).unsqueeze(0).to(device)
            with torch.no_grad():
                image = model.encode_image(image).float()
                image /= image.norm(dim=-1, keepdim=True)
            curr_vid.append(image)
        res.append(curr_vid)
    return res

In [32]:
def search_preprocessed_data(data, text):
    res = []
    text = clip.tokenize(text).to(device)
    with torch.no_grad():
        text = model.encode_text(text).float()
    text /= text.norm(dim=-1, keepdim=True)
    i = 0
    for vid in data:
        score = []
        for frame in vid:
            similarity = text.cpu().numpy() @ frame.cpu().numpy().T
            score.append(similarity)
        if(check(score)):
            res.append(i)
        i += 1
    return res

In [93]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast
app = Flask(__name__)
api = Api(app)

In [92]:
video_paths = [r"C:\Users\thein\Downloads\v.mp4"]
links = ["https://www.youtube.com/"]
data = process_videos(video_paths)
d = {}
i = 0
for vid in links:
    d[i] = vid
    i+=1

In [94]:
class clipSearch(Resource):
    def get(self):
        parser = reqparse.RequestParser()
        parser.add_argument('text', required=True)
        args = parser.parse_args()
        text = args['text']
        ans = []
        res = search_preprocessed_data(data, text)
        for vid in res:
            ans.append(d[vid])
        return {'data': ans}, 
api.add_resource(clipSearch, '/clipsearch') 

In [95]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Mar/2023 17:18:58] "GET /clipsearch?text=hi HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2023 17:19:03] "GET /clipsearch?text=bye HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2023 17:19:10] "GET /clipsearch?text=sample HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2023 17:19:15] "GET /clipsearch?text=text HTTP/1.1" 200 -
